# RSnowflake -- Workspace Notebook Test Suite

This notebook tests the **RSnowflake** package (pure R DBI connector for Snowflake)
inside a Snowflake Workspace Notebook.

**Key things being validated:**
1. Session token auto-detection (`SNOWFLAKE_TOKEN`)
2. Account auto-detection (`SNOWFLAKE_ACCOUNT` / `SNOWFLAKE_HOST` / reticulate fallback)
3. DBI operations: connect, query, write, read, disconnect
4. Type roundtrips: integer, double, character, logical, Date, POSIXct
5. Token refresh on long-running sessions

**Sections:**
1. Setup (install R + RSnowflake)
2. Connect via session token
3. Scalar queries & type mapping
4. Table operations (write, read, append, remove)
5. dbSendQuery / dbFetch / dbClearResult
6. Quoting & identifiers
7. Cleanup

## 1. Setup

Install the R environment (if not already installed) and register the `%%R` magic.

In [ ]:
# Install R + rpy2 via setup script (included in this directory)
!bash setup_r_environment.sh --basic

In [ ]:
from r_helpers import setup_r_environment
setup_r_environment()

### Install RSnowflake dependencies and package

Install the hard dependencies from CRAN, then install RSnowflake from the
bundled source tarball (or from a git stage). Adjust the path below if needed.

In [ ]:
%%R
# Install RSnowflake dependencies from CRAN
pkgs <- c("DBI", "httr2", "jsonlite", "rlang", "cli")
for (pkg in pkgs) {
  if (!requireNamespace(pkg, quietly = TRUE)) {
    install.packages(pkg, repos = "https://cloud.r-project.org")
  }
}
cat("Dependencies installed.\n")

In [ ]:
%%R
# Install RSnowflake from the repo source.
# This notebook lives at RSnowflake/inst/notebooks/, so "../.." is the package root.
pkg_dir <- normalizePath(file.path(getwd(), "..", ".."))
cat("Installing RSnowflake from:", pkg_dir, "\n")
install.packages(pkg_dir, repos = NULL, type = "source")

## 2. Connect via Session Token

In a Workspace Notebook, the `SNOWFLAKE_TOKEN` environment variable is
automatically set. RSnowflake detects this and uses it for authentication.
The account is resolved from `SNOWFLAKE_ACCOUNT`, `SNOWFLAKE_HOST`, or
by querying the active Snowpark session via reticulate.

In [ ]:
%%R
library(RSnowflake)
library(DBI)

# Verify session token sources
cat("SNOWFLAKE_TOKEN env var:", nzchar(Sys.getenv("SNOWFLAKE_TOKEN", "")), "\n")
cat("/snowflake/session/token file:", file.exists("/snowflake/session/token"), "\n")
cat("SNOWFLAKE_ACCOUNT:", Sys.getenv("SNOWFLAKE_ACCOUNT", "(not set)"), "\n")
cat("SNOWFLAKE_HOST:", Sys.getenv("SNOWFLAKE_HOST", "(not set)"), "\n")

In [ ]:
%%R
# Connect -- no explicit account/user/token needed in Workspace!
con <- dbConnect(Snowflake())

# Verify
cat("Valid:", dbIsValid(con), "\n")
print(con)

# Show connection info
info <- dbGetInfo(con)
cat("Host:", info$host, "\n")
cat("Database:", info$dbname, "\n")
cat("Warehouse:", info$warehouse, "\n")

## 3. Scalar Queries & Type Mapping

Test that basic Snowflake types map correctly to R types.

In [ ]:
%%R
tests_passed <- 0L
tests_failed <- 0L

check <- function(name, condition) {
  if (isTRUE(condition)) {
    cat("  PASS:", name, "\n")
    tests_passed <<- tests_passed + 1L
  } else {
    cat("  FAIL:", name, "\n")
    tests_failed <<- tests_failed + 1L
  }
}

cat("== Scalar Queries ==\n")

df <- dbGetQuery(con, "SELECT 42 AS int_col")
check("integer column", is.integer(df$INT_COL) && df$INT_COL == 42L)

df <- dbGetQuery(con, "SELECT 3.14::DOUBLE AS dbl_col")
check("double column", is.double(df$DBL_COL) && abs(df$DBL_COL - 3.14) < 0.01)

df <- dbGetQuery(con, "SELECT 'hello' AS str_col")
check("string column", is.character(df$STR_COL) && df$STR_COL == "hello")

df <- dbGetQuery(con, "SELECT TRUE AS bool_col")
check("boolean column", is.logical(df$BOOL_COL) && df$BOOL_COL == TRUE)

df <- dbGetQuery(con, "SELECT '2024-06-15'::DATE AS date_col")
check("date column", inherits(df$DATE_COL, "Date"))

df <- dbGetQuery(con, "SELECT '2024-06-15 10:30:00'::TIMESTAMP_NTZ AS ts_col")
check("timestamp column", inherits(df$TS_COL, "POSIXct"))

df <- dbGetQuery(con, "SELECT NULL::INTEGER AS null_col")
check("NULL returns NA", is.na(df$NULL_COL))

## 4. Table Operations

Test write, read, append, list, and remove operations.

In [ ]:
%%R
cat("== Table Operations ==\n")

# Lowercase column names -- RSnowflake preserves case via quoted identifiers
test_df <- data.frame(
  id = 1:5,
  name = c("Alice", "Bob", "Carol", "Dave", "Eve"),
  score = c(95.5, 87.3, 92.1, 78.9, 88.4),
  active = c(TRUE, FALSE, TRUE, TRUE, FALSE),
  stringsAsFactors = FALSE
)

# Write
dbWriteTable(con, "RSNOWFLAKE_TEST", test_df, overwrite = TRUE)
check("dbWriteTable succeeds", TRUE)

# Exists
check("dbExistsTable finds it", dbExistsTable(con, "RSNOWFLAKE_TEST"))

# List tables
tables <- dbListTables(con)
check("dbListTables includes RSNOWFLAKE_TEST",
      "RSNOWFLAKE_TEST" %in% toupper(tables))

# List fields -- column names round-trip in their original case
fields <- dbListFields(con, "RSNOWFLAKE_TEST")
check("dbListFields returns column names", length(fields) == 4)
check("dbListFields preserves column case",
      identical(fields, c("id", "name", "score", "active")))
cat("  Fields:", paste(fields, collapse = ", "), "\n")

# Read
df_read <- dbReadTable(con, "RSNOWFLAKE_TEST")
check("dbReadTable returns 5 rows", nrow(df_read) == 5)
check("dbReadTable integer roundtrip", is.integer(df_read$id))
check("dbReadTable string roundtrip", df_read$name[1] == "Alice")

# Append
extra <- data.frame(id = 6L, name = "Frank", score = 91.0, active = TRUE)
dbWriteTable(con, "RSNOWFLAKE_TEST", extra, append = TRUE)
df_after <- dbReadTable(con, "RSNOWFLAKE_TEST")
check("dbWriteTable append", nrow(df_after) == 6)

## 5. dbSendQuery / dbFetch / dbClearResult

Test the streaming result set workflow.

In [ ]:
%%R
cat("== dbSendQuery / dbFetch / dbClearResult ==\n")

sql <- 'SELECT * FROM RSNOWFLAKE_TEST ORDER BY "id"'
res <- dbSendQuery(con, sql)
check("dbSendQuery returns SnowflakeResult", is(res, "SnowflakeResult"))
check("Result is valid", dbIsValid(res))

df <- dbFetch(res)
check("dbFetch returns data.frame", is.data.frame(df))
check("dbFetch returns 6 rows", nrow(df) == 6)

# S4 mutability: second fetch should return empty
df2 <- dbFetch(res)
check("Second dbFetch returns 0 rows (mutability fix)", nrow(df2) == 0)

check("dbHasCompleted", dbHasCompleted(res))
check("dbGetStatement", dbGetStatement(res) == sql)
check("dbColumnInfo", nrow(dbColumnInfo(res)) == 4)

dbClearResult(res)
check("Result cleared", !dbIsValid(res))

## 6. Quoting & Identifiers

In [ ]:
%%R
cat("== Quoting ==\n")

check("dbQuoteIdentifier",
      as.character(dbQuoteIdentifier(con, "my_table")) == '"my_table"')

check("dbQuoteString",
      as.character(dbQuoteString(con, "it's")) == "'it''s'")

check("dbQuoteString NA",
      as.character(dbQuoteString(con, NA_character_)) == "NULL")

check("dbQuoteLiteral int",
      as.character(dbQuoteLiteral(con, 42L)) == "42")

check("dbQuoteLiteral date",
      grepl("DATE", as.character(dbQuoteLiteral(con, as.Date("2024-01-15")))))

## 7. DML & dbExecute

In [ ]:
%%R
cat("== DML ==\n")

affected <- dbExecute(con, 'DELETE FROM RSNOWFLAKE_TEST WHERE "id" = 6')
check("dbExecute DELETE returns affected", affected >= 1)

df <- dbGetQuery(con, "SELECT COUNT(*) AS cnt FROM RSNOWFLAKE_TEST")
check("Row deleted", df$CNT == 5)

## 8. Cleanup & Summary

In [ ]:
%%R
cat("== Cleanup ==\n")

dbRemoveTable(con, "RSNOWFLAKE_TEST")
check("Table removed", !dbExistsTable(con, "RSNOWFLAKE_TEST"))

dbDisconnect(con)
check("Disconnected", !dbIsValid(con))

cat(sprintf("\n== Results: %d passed, %d failed ==\n",
    tests_passed, tests_failed))